# Topas Work - Team protons and heavy ions
### PHY-7051 Physique des radiations en radiothérapie et en radiologie
Gabrielle Germain (111 266 117), Hanieh Sadat Jozi (111 XXX XXX), Philippe Dionne (111 XXX XXX)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Useful functions

In [ ]:
def csv_to_data(path:str, skiprows:int=9, delimiter:str=', ') -> tuple:
    """ Reads the csv file and converts it to a tuple containing 3 columns matrices

    Args:
        path (str): absolute path on the computer. Must have the format r"C:\path.csv"
        skiprows (int, optional): Rows to skip the header. Defaults to 9.
        delimiter (str, optional): Defaults to ', '.

    Returns:
        tuple: (bin, dose, standard-deviation)
    """

    data = np.loadtxt(path, skiprows=skiprows, delimiter=delimiter, usecols=(2, 3, 4))
    bin = data[:, 0]
    dose = data[:, 1]
    std = data[:, 2]
    return (bin, dose, std)

Constants

## Central-axis depth-dose distribution in water of a monoenergetic beam of protons or carbon ions

In [ ]:
""" Le code ci-bas génère la figure 6.12 du manuel de référence, soit [...] dans un fantôme d'eau."""

# ------------------------------- Paramètres ------------------------------------------- #
""" Initialisation des paramètres utilisés dans les codes de simulation TOPAS. """

field_size = 100 # cm2
energies_p = np.array([122, 183, 230])      # MeV
energies_c = np.array([220, 350, 430])      # MeV/u, u = 12
nbr_p = np.array([1000000, 1000000, 1000000]) # nbr de protons simulés pour chaque énergie
nbr_c = np.array([100000, 100000, 100000]) # nbr d'ions de carbone simulés pour chaque énergie
bin_nbr = np.array([350, 350]) # nbr de bins pour chaque particule
tot_depths = np.array([35, 35]) # profondeur des scorers pour chaque particule, en cm
in_fluence_p = nbr_p/field_size # cm-2
in_fluence_c = nbr_c/field_size # cm-2

np.seterr(divide='ignore', invalid='ignore')

# initialisation des données
std_rel_p = np.empty(np.size(energies_p))
R50_p = np.empty(np.size(energies_p))
Dmax_p = np.empty(np.size(energies_p))

std_rel_c = np.empty(np.size(energies_c))
R50_c = np.empty(np.size(energies_c))
Dmax_c = np.empty(np.size(energies_c))

# initialisation des figures
fig1 = plt.figure(figsize=(10, 15))
ax1 = plt.subplot(2, 1, 1)
plt.xlabel("Profondeur dans l'eau, [cm]", fontsize='18')
plt.ylabel("Énergie déposée [MeV cm$^2$/g]", fontsize='18')
plt.title("a)",  fontsize='20')
plt.tick_params(direction='in',labelsize='14')
ax2 = plt.subplot(2, 1, 2)
plt.xlabel("Profondeur dans l'eau [cm]", fontsize='18')
plt.ylabel("Énergie déposée [MeV cm$^2$/g]", fontsize='18')
plt.title("b)",  fontsize='20')
plt.tick_params(direction='in',labelsize='14')

# marker = ['-','--','-.',':','k-','k--','k-.','k:']

# ------------------------------- Fonctions de calcul ------------------------------------------- #
## Protons en premier
for i in range (len(energies_p)):
    energy = energies_p[i]
    path = r"Topas_project\Figure6_12\6_12_protons_{}MeV_broad.csv".format(energy)
    bins, dose, std = csv_to_data(path)



## Depth of the Bragg Peak for a carbon ions beam

## Beam quality of a pristine and modulated beam of proton